In [2]:
### CREATING RANDOMIZATION.CSV
import pandas as pd
import numpy as np
import random
import csv
#mypath = ('C:/Users/luca.chech.16/Desktop/Random_probes_last/Luca/')
mypath = 'C:/Users/KeK/OneDrive - University College London/Random_probes_last/Luca/'

df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')


df.reset_index(drop=False)

image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}

# DEFINING TARGET-PRESENT PROBES
lista= possible_images_dict.keys()
target_present = []
intermediate = []
categories = [' bottle', ' horse', ' pottedplant', ' dog',
              ' cat', ' person', ' aeroplane', ' car', ' chair', ' sofa', ' bird', ' boat']
p_load = ['low','high']

for category in categories:
    for load in p_load:
        while True:
            trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
            if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                and trial not in intermediate and trial not in target_present:
                intermediate.append(trial)
            if len(intermediate) == 1:
                for each in intermediate:
                    target_present.append(each)
                    possible_images_dict[each].append(category)
                intermediate = []
                break

for n in target_present:
    possible_images_dict[n].append('Target Present')
# DEFINING TARGET-ABSENT PROBES
target_absent = []
intermediate = []
for category in categories:
    for load in p_load:
        while True:
            trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
            if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                and trial not in intermediate and trial not in target_absent and trial not in target_present:
                intermediate.append(trial)
            if len(intermediate) == 1:
                for each in intermediate:
                    target_absent.append(each)
                intermediate = []
                break
for each in target_absent:
    absent_list = [category for category in categories if category not in possible_images_dict[each]]
    absent_question = str(np.random.choice(absent_list, 1, replace= False))[2:-2]
    possible_images_dict[each].append(absent_question)
    possible_images_dict[each].append('Target Absent')
    print possible_images_dict[each]

for image in lista:
    if image not in target_present and image not in target_absent:
        possible_images_dict[image].append('NA')
        possible_images_dict[image].append('No Target')

# DEFINING CRITICAL TRIALS
critical_trials = []
def critical_trial(load_level):
    global critical_trials
    temp = []
    container = []
    for element in lista:
        if load_level in possible_images_dict[element]:
            container.append(element)
    for category in categories:
        while len(temp) <3 :
            intermediate = str(np.random.choice(container, 1, replace= False))[2:-2]
            if category in possible_images_dict[intermediate] \
                and intermediate not in temp and intermediate not in critical_trials:
                temp.append(intermediate)
        for each in temp:
            critical_trials.append(each)
        temp =[]
    print len(critical_trials)
         
critical_trial('low')
critical_trial('high')

print len(critical_trials) == len(set(critical_trials))
for each in critical_trials:
    possible_images_dict[each].append('Critical')
for each in possible_images_dict:
    if 'Critical' not in possible_images_dict[each]:
        possible_images_dict[each].append('Normal')

df = pd.DataFrame.from_dict(possible_images_dict)
df= df.T
df.columns = ['Cat.1','Cat.2','Cat.3','Cat.4','Cat.5','TS','Load','exemplar','question','present_absent','trial_type']
df.TS = df.TS.astype('float')
df.to_csv(mypath + 'randomization_15.csv')


#Defininfg blocks 
probes_per_block = []
critical_per_block = []
normal_per_block = []
present_used = []
absent_used = []
critical_used = []
normal_no_probe = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
normal_no_probe_used = []
block = []
to_insert = []
print len(normal_no_probe)

#%pdb

def block_low():
# Pick 3 present probes and 3 absent probes
    global critical_per_block,probes_per_block, present_used, absent_used, critical_used, normal_per_block, to_insert
    while len(probes_per_block) < 3:
        probe = str(np.random.choice(target_present,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in present_used and 'low' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    while 3 <= len(probes_per_block) < 6:
        probe = str(np.random.choice(target_absent,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in absent_used and 'low' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    for n in probes_per_block[0:3]:
        present_used.append(n)
    for n in probes_per_block[3:]:
        absent_used.append(n)
#------------------------------------------------------------------------------    
#Counting critical probes    
    critical_probes_counter = 0
    for n in probes_per_block:
        if 'Critical' in possible_images_dict[n]:
            critical_probes_counter = critical_probes_counter + 1
    #for n in probes_per_block:
    #    print possible_images_dict[n]
    print 'Present Used: ',len(present_used)
    print 'Absent Used ',len(absent_used)
    print 'Critical probes counter: ',critical_probes_counter
#------------------------------------------------------------------------------
#Picking critical trials
    while len(critical_per_block) < (9-critical_probes_counter):
        image = str(np.random.choice(critical_trials,1,replace = False))[2:-2]
        if image not in critical_per_block and image not in critical_used \
            and 'No Target' in possible_images_dict[image] and 'low' in possible_images_dict[image]:
            critical_per_block.append(image) 
    for n in critical_per_block:
        critical_used.append(n)
        print possible_images_dict[n]
    #temp = []
#------------------------------------------------------------------------------
#Picking normal trials
    while len(normal_per_block) < 54 - (9 - critical_probes_counter):
        image = str(np.random.choice(normal_no_probe,1,replace = False))[2:-2]
        if image not in normal_per_block and image not in normal_no_probe_used and 'low' in possible_images_dict[image]:
            normal_per_block.append(image)
    for n in normal_per_block:
        normal_no_probe_used.append(n)
    print 'len(normal_no_probe_used): ',len(normal_no_probe_used)
#-----------------------------------------------------------------------------
    to_insert = probes_per_block + critical_per_block
    print 'len(to_insert): ',len(to_insert)

    #Adding critical (no probe) trials to the block
    def constrained_sum_sample_pos_adding_criticals(n, total):
        """Return a randomly chosen list of n positive integers summing to total.
        Each such list is equally likely to occur."""
        global block, critical_per_block, new_list, diff, to_insert
        liste_scartate = 0
        while True:
            global sequences_length
            dividers = sorted(random.sample(xrange(1, total), n - 1))
            sequences_length = [a - b for a, b in zip(dividers + [total], [0] + dividers)]
            acc = 0
            new_list = []
            for i in sequences_length:
                acc = acc + i
                new_list.append(acc - 1)
            diff = []
            for i in range(len(to_insert)):
                diff.append(new_list[i+1] - new_list[i])
            liste_scartate = liste_scartate + 1
            if 1 not in diff and 2 not in diff:
                print 'Liste scartate: ',liste_scartate
                print 'New List: ', new_list
                block = normal_per_block
                #print len(block)
                random.shuffle(to_insert)
                for i in range(len(to_insert)):
                    block.insert(new_list[i], to_insert[i])
                new_list = [n for n in new_list if 'No Target' not in possible_images_dict[block[n]]]
                print new_list
                print len(new_list)
                return  block, new_list
                break
#-----------------------------------------------------------------------------    
    print 'len(block): ',len(block)
    constrained_sum_sample_pos_adding_criticals(15,60)
    print 'len(block): ',len(block)
#print new_list
#for n in range(len(new_list)):
#    print new_list[n], possible_images_dict[block[new_list[n]]]
#print diff
    probes_per_block = []
    critical_per_block = []
    normal_per_block = []
    to_insert = []
    print 'Normal used:',len(normal_no_probe_used)
    print'---------------------------------------------------'
    return block, new_list
    

def block_high():
# Pick 3 present probes and 3 absent probes
    global critical_per_block,probes_per_block, present_used, absent_used, critical_used, normal_per_block, to_insert
    while len(probes_per_block) < 3:
        probe = str(np.random.choice(target_present,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in present_used and 'high' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    while 3 <= len(probes_per_block) < 6:
        probe = str(np.random.choice(target_absent,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in absent_used and 'high' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    for n in probes_per_block[0:3]:
        present_used.append(n)
    for n in probes_per_block[3:]:
        absent_used.append(n)
#------------------------------------------------------------------------------    
#Counting critical probes    
    critical_probes_counter = 0
    for n in probes_per_block:
        if 'Critical' in possible_images_dict[n]:
            critical_probes_counter = critical_probes_counter + 1
    #for n in probes_per_block:
    #    print possible_images_dict[n]
    print 'Present Used: ',len(present_used)
    print 'Absent Used ',len(absent_used)
    print 'Critical probes counter: ',critical_probes_counter
#------------------------------------------------------------------------------
#Picking critical trials
    while len(critical_per_block) < (9-critical_probes_counter):
        image = str(np.random.choice(critical_trials,1,replace = False))[2:-2]
        if image not in critical_per_block and image not in critical_used \
            and 'No Target' in possible_images_dict[image] and 'high' in possible_images_dict[image]:
            critical_per_block.append(image) 
    for n in critical_per_block:
        critical_used.append(n)
        print possible_images_dict[n]
    #temp = []
#------------------------------------------------------------------------------
#Picking normal trials
    while len(normal_per_block) < 54 - (9 - critical_probes_counter):
        image = str(np.random.choice(normal_no_probe,1,replace = False))[2:-2]
        if image not in normal_per_block and image not in normal_no_probe_used and 'high' in possible_images_dict[image]:
            normal_per_block.append(image)
    for n in normal_per_block:
        normal_no_probe_used.append(n)
    print 'len(normal_no_probe_used): ',len(normal_no_probe_used)
#-----------------------------------------------------------------------------
    to_insert = probes_per_block + critical_per_block
    print 'len(to_insert): ',len(to_insert)

    #Adding critical (no probe) trials to the block
    def constrained_sum_sample_pos_adding_criticals(n, total):
        """Return a randomly chosen list of n positive integers summing to total.
        Each such list is equally likely to occur."""
        global block, critical_per_block, new_list, diff, to_insert
        liste_scartate = 0
        while True:
            global sequences_length
            dividers = sorted(random.sample(xrange(1, total), n - 1))
            sequences_length = [a - b for a, b in zip(dividers + [total], [0] + dividers)]
            acc = 0
            new_list = []
            for i in sequences_length:
                acc = acc + i
                new_list.append(acc - 1)
            diff = []
            for i in range(len(to_insert)):
                diff.append(new_list[i+1] - new_list[i])
            liste_scartate = liste_scartate + 1
            if 1 not in diff and 2 not in diff:
                print 'Liste scartate: ',liste_scartate
                print 'New List: ', new_list
                block = normal_per_block
                #print len(block)
                random.shuffle(to_insert)
                for i in range(len(to_insert)):
                    block.insert(new_list[i], to_insert[i])
                new_list = [n for n in new_list if 'No Target' not in possible_images_dict[block[n]]]
                print new_list
                print len(new_list)
                return  block, new_list
                break
#-----------------------------------------------------------------------------    
    print 'len(block): ',len(block)
    constrained_sum_sample_pos_adding_criticals(15,60)
    print 'len(block): ',len(block)
#print new_list
#for n in range(len(new_list)):
#    print new_list[n], possible_images_dict[block[new_list[n]]]
#print diff
    probes_per_block = []
    critical_per_block = []
    normal_per_block = []
    to_insert = []
    print 'Normal used:',len(normal_no_probe_used)
    print'---------------------------------------------------'
    return block, new_list
    

low_one, lista_low_one = block_low()
low_two, lista_low_two = block_low()
low_three, lista_low_three = block_low()
low_four, lista_low_four = block_low()
high_one, lista_high_one = block_high()
high_two, lista_high_two = block_high()
high_three, lista_high_three = block_high()
high_four, lista_high_four = block_high()

print len(normal_no_probe)
print len(normal_no_probe_used)
print len(normal_no_probe) == len(set(normal_no_probe_used))
print len(target_present)
print len(present_used)
print len(target_present) == len(set(present_used))
print len(target_absent)
print len(absent_used)
print len(target_absent) == len(set(absent_used))
print len(critical_trials)
print len(critical_used)
critical_probes_counter = 0
probes = target_absent + target_present
for probe in probes:
    if 'Target Present' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
        critical_probes_counter = critical_probes_counter +1
    if 'Target Absent' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
        critical_probes_counter = critical_probes_counter +1
    
print critical_probes_counter



block_dict = {}
block_dict['low_one'] = low_one
block_dict['low_two'] = low_two
block_dict['low_three'] = low_three
block_dict['low_four'] = low_four
block_dict['high_one'] = high_one
block_dict['high_two'] = high_two
block_dict['high_three'] = high_three
block_dict['high_four'] = high_four

block_dict['low_one']
print len(block_dict['low_one'])
print low_one

[' chair', ' diningtable', ' bottle', nan, nan, 20.038119878562025, 'low', ' bottle', ' bird', 'Target Absent']
[' person', ' chair', ' bottle', ' diningtable', nan, 35.134856512845488, 'high', ' bottle', ' boat', 'Target Absent']
[' horse', nan, nan, nan, nan, 20.9577483152902, 'low', ' horse', ' aeroplane', 'Target Absent']
[' person', ' horse', nan, nan, nan, 31.969634324354409, 'high', ' horse', ' bird', 'Target Absent']
[' pottedplant', nan, nan, nan, nan, 11.902212339033813, 'low', ' pottedplant', ' boat', 'Target Absent']
[' pottedplant', ' sofa', ' chair', ' tvmonitor', nan, 29.464580095555625, 'high', ' pottedplant', ' person', 'Target Absent']
[' dog', nan, nan, nan, nan, 17.102590259239356, 'low', ' dog', ' chair', 'Target Absent']
[' pottedplant', ' person', ' chair', ' dog', nan, 34.099790178372324, 'high', ' dog', ' bottle', 'Target Absent']
[' cat', nan, nan, nan, nan, 20.60608553782437, 'low', ' cat', ' aeroplane', 'Target Absent']
[' person', ' cat', nan, nan, nan, 29.

IndexError: list index out of range